![](img/572_banner.png)

# Lab 4: Advanced Deep Learning - Transfer Learning and GANs

**Tomas Beuzen, January 2021**

Pick your poison! In this week's lab I want you to solve a real-world project using deep learning. The lab is not long, but I think it's fun, and I'll be getting you to upload this lab to Kaggle so that you can leverage Kaggle's GPUs to speed up our deep learning. I've presented you with two "mini-projects", both of which I've had to solve before in research and industry settings. You only have to do one of these mini-projects (but feel free to do both - we will only grade one, so you won't get double marks, but you might feel twice as good about yourself!). The projects are:
1. **Transfer Learning**: build a CNN to classify coastal satellite imagery as containing buildings or not.
2. **Generative Adversarial Network (GAN)**: build a GAN to generate synthetic coastal satellite imagery.

You'll need to use all the powers you've learned! Use the ~~force~~ Torch!

![](img/force.png)

## Table of Contents
<hr>

<div class="toc"><ul class="toc-item"><li><span><a href="#Instructions" data-toc-modified-id="Instructions-2">Instructions</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-3">Imports</a></span></li><li><span><a href="#Getting-Started-with-Kaggle" data-toc-modified-id="Getting-Started-with-Kaggle-4">Getting Started with Kaggle</a></span></li><li><span><a href="#Mini-Project-1:-Transfer-Learning" data-toc-modified-id="Mini-Project-1:-Transfer-Learning-5">Mini Project 1: Transfer Learning</a></span></li><li><span><a href="#Mini-Project-2:-Generative-Adversarial-Networks" data-toc-modified-id="Mini-Project-2:-Generative-Adversarial-Networks-6">Mini Project 2: Generative Adversarial Networks</a></span></li><li><span><a href="#(Optional)-Exercise-3:-Setting-Priorities-and-Managing-your-Time" data-toc-modified-id="(Optional)-Exercise-3:-Setting-Priorities-and-Managing-your-Time-7">(Optional) Exercise 3: Setting Priorities and Managing your Time</a></span></li><li><span><a href="#Submit-to-Canvas-and-GitHub" data-toc-modified-id="Submit-to-Canvas-and-GitHub-8">Submit to Canvas and GitHub</a></span></li></ul></div>

## Instructions
<hr>

rubric={mechanics:3}

**Link to your GitHub repository:**

You will receive marks for correctly submitting this assignment. To submit this assignment you should:

1. Push your assignment to your GitHub repository!
2. Provide a link to your repository in the space provided above.
2. Upload a HTML render of your assignment to Canvas. The last cell of this notebook will help you do that.
3. Be sure to follow the [General Lab Instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions/). You can view a description of the different rubrics used for grading in MDS [here](https://github.com/UBC-MDS/public/tree/master/rubric).

Here's a break down of the required and optional exercises in this lab:

|         | Number of Exercises | Points |
|:-------:|:-------------------:|:------:|
| Mechanics| 1 | 3 |
| Required| 2 | 47 each<br>(only one exercise needs to be completed) |
| Optional| 1  | 1000000 |

## Imports
<hr>

In [ ]:
import numpy as np
import pandas as pd
from collections import OrderedDict
import torch
from torch import nn, optim
from torchvision import datasets, transforms, utils, models
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from PIL import Image
plt.style.use('ggplot')
plt.rcParams.update({'font.size': 16, 'axes.labelweight': 'bold', 'axes.grid': False})
# from canvasutils.submit import submit, convert_notebook  # this won't import in kaggle, so comment it back when you're working locally again

## Getting Started with Kaggle
<hr>

We are going to run this notebook on the cloud using [Kaggle](https://www.kaggle.com). Kaggle offers 30+ hours of free GPU usage per week which should be much more than adequate for this lab. To get started, follow these steps:

1. Go to https://www.kaggle.com/kernels
2. Make an account if you don't have one
3. Select `+ New Notebook`
4. Go to `File -> Upload`
5. Upload this notebook
6. On the right-hand side of your Kaggle notebook, make sure:
    - Make sure `Internet` is enabled.
    - In the `Accelerator` dropdown, choose `GPU` when you're ready to use it (feel free to turn it on/off as you need it).
    
Once you've done all your work on Kaggle, you can download the notebook from Kaggle. That way any work you did on Kaggle won't be lost.

## Mini Project 1: Transfer Learning
<hr>

rubric={accuracy:47}

>Remember: you only have to do this project **OR** Mini Project 2. You do not have to do both.

In this exercise you're going to practice transfer learning. We're going to develop a model that can determine if a satellite image of a coastal landscape contains man-made structures ("developed") or not ("natural"). Identifying areas of coastline that have structures on them is important for a range of purposes:
- Identifying areas at high risk of storm damage;
- Determining areas at risk of sea level rise;
- Evaluating human impact on coastal ecology;
- etc.

Below is an example of a "natural coastline" (note that this image won't show up in Kaggle, it's a local file):

![](img/natural-coast.jpg)

And here is a "developed coastline":

![](img/developed-coast.jpg)

I've uploaded a dataset of 2186 images of both developed and natural coastlines to Kaggle. To access it from your Kaggle notebook:
1. Click `+ Add data` at the top right of the notebook.
2. Search for "DSCI572-satellite-transfer" and click `Add`
3. This will add the data to your notebook after a restart

### 1.1: CNN from Scratch

In this exercise, you should build a binary classification CNN from scratch that can identify "developed" and "natural" coastlines, i.e., your model will accept an image as input and will classify it as "developed" or "natural".

In Kaggle, running the follow cell should print out `"Using device: cuda"` which means a GPU is available:

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device.type}")

To make use of the GPU, you should:
1. Move your model to the GPU after creating it with the syntax:

```python
model.to(device)
```

2. In your training/validation loops, each batch should be moved to the GPU using syntax like:

```python
for X, y in dataloader:
    X, y = X.to(device), y.to(device)
    ...
```

3. If you're having trouble, check out how I did this in Lecture 7 or 8.

Here are some guidelines for building your binary classification CNN from scratch:

- You may use any architecture you like.
- This is the path to the data in your notebook:
    - Training data: `"../input/dsci572satellitetransfer/labelled/train/"`.
    - Training data: `"../input/dsci572satellitetransfer/labelled/validation/"`.
- You should use an `IMAGE_SIZE = 200` pixels in your data loader (the raw images are 400 x 400 pixels, we don't need them that big).
- **You must train your model for at least 20 epochs and print or plot the accuracy on the validation data for us to see.**

>If you want to take a look at the images after making a `train_loader`, try this code:

```python
# Plot samples
sample_batch = next(iter(train_loader))
plt.figure(figsize=(10, 8)); plt.axis("off"); plt.title("Sample Training Images")
plt.imshow(np.transpose(utils.make_grid(sample_batch[0], padding=1, normalize=True),(1, 2, 0)));
```

In [ ]:
# Your solution goes here.

### 1.2: Feature Extractor

Okay, nice job! Now, in this exercise, you should leverage a pre-trained model customised with your own layer(s) on top (like we did in [Lecture 7 here](https://pages.github.ubc.ca/MDS-2020-21/DSCI_572_sup-learn-2_students/lectures/lecture7_cnns-pt2.html#feature-extractor)), to build a binary classifier CNN that can identify "developed" and "natural" coastlines:
- You can use any model you wish, but I recommend using `densenet` as we did in lecture.
- Train your model for at least 20 epochs.
- Print or plot the accuracy on the validation data.
- Comment on how this model did compared to your "from scratch" model.

In [ ]:
# Your solution goes here.

### 1.3: Fine Tuning

Alrighty now we're feeling powerful! In this final exercise, you should fine tune your model above by updating all or some of the layers during training. Refer to [these lecture notes](https://pages.github.ubc.ca/MDS-2020-21/DSCI_572_sup-learn-2_students/lectures/lecture7_cnns-pt2.html#fine-tuning) for help with approaching this problem:
- You can fine tune as many layers as you like, I recommend just fine tuning the whole model. We have a GPU, so why not!
- Train your model for at least 20 epochs.
- Print or plot the accuracy on the validation data.
- Comment on how this model did compared to your "from scratch" and "feature extractor" models.

In [ ]:
# Your solution goes here.

## Mini Project 2: Generative Adversarial Networks
<hr>

rubric={accuracy:47}

>Remember: you only have to do this project **OR** Mini Project 1. You do not have to do both.

In this exercise you're going to practice building a generative adversarial network (GAN). GANs are data generating models. Collecting high resolution imagery of coastal landscapes is expensive and time-consuming, yet such data is critical for research and planning. Imagine if we could generate realistic coastal satellite data on demand using a GAN? One application I've used GANs for is to simulate storm impacts on coastlines - big storms happen only a few times a year and collecting imagery of post-storm impacts is expensive and time-sensitive. The ability to generate images of storm-impacted coastlines allows us to hypothesise and plan for potential impact scenarios - pretty cool!

I will say that GANs are incredibly hard to train - we don't have a lot of data and you may not get good results in this exericse, but it's fun to try! Here's an example of what I came up with:

![](img/gan-coast.gif)

Okay, let's get to work. I've uploaded a dataset of 2186 coastal satellite images to Kaggle. To access it from your Kaggle notebook:
1. Click `+ Add data` at the top right of the notebook.
2. Search for "DSCI572-satellite-gan" and click `Add`
3. This will add the data to your notebook after a restart

### 2.1: Preparing the Data

In Kaggle, running the follow cell should print out `"Using device: cuda"` which means a GPU is available:

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device.type}")

To make use of the GPU, you should:
- Move your model to the GPU after creating it with the syntax:

```python
model.to(device)
```

- In your training loop, each batch should be moved to the GPU using syntax like:

```python
for X, _ in dataloader:
    X = X.to(device)
    ...
```

- Note above that we don't need the label for training a GAN, so I ignore it by un-packing it into an underscore `_` (which is typically Python convention).
- If you're having trouble, check out how I did this in Lecture 7 or 8.

Okay, prepare the data by creating a `data_loader`, here are some guidelines for you:

- This is the path to the data in your notebook: `"../input/dsci572satellitegan/"`.
- You should use an `IMAGE_SIZE = 200` pixels in your data loader (the raw images are 400 x 400 pixels, we don't need them that big).

>If you want to take a look at the images after making a `data_loader`, try this code:

```python
# Plot samples
sample_batch = next(iter(data_loader))
plt.figure(figsize=(10, 8)); plt.axis("off"); plt.title("Sample Training Images")
plt.imshow(np.transpose(utils.make_grid(sample_batch[0], padding=1, normalize=True),(1, 2, 0)));
```

In [ ]:
# Your solution goes here.

### 2.2: Create the Generator

Now, we need to create a generator for our GAN. You may reuse/modify the code from [Lecture 8](https://pages.github.ubc.ca/MDS-2020-21/DSCI_572_sup-learn-2_students/lectures/lecture8_advanced-deep-learning.html#generative-adversarial-networks-gans), or build your own. You should use a `LATENT_SIZE=100` for the generator.

In [ ]:
# Your solution goes here.

### 2.3: Create the Discriminator

Now, we need to create a discriminator for our GAN. You may reuse/modify the code from [Lecture 8](https://pages.github.ubc.ca/MDS-2020-21/DSCI_572_sup-learn-2_students/lectures/lecture8_advanced-deep-learning.html#generative-adversarial-networks-gans), or build your own.

In [ ]:
# Your solution goes here.

### 2.4: Initialize Weights

GANs can be quite sensitive to the initial weights assigned to each layer when we instantiate the model. Instantiate your generator and discriminator and then specify their initial weights as follows:
- `Conv2d()` layers: normal distribution with `mean=0.0` and `std=0.02`
- `ConvTranspose2d()` layers: normal distribution with `mean=0.0` and `std=0.02`
- `BatchNorm2d()` layers: normal distribution with `mean=1.0` and `std=0.02` for the weights, zeroes for the biases

>Hint: refer to [Lecture 8](https://pages.github.ubc.ca/MDS-2020-21/DSCI_572_sup-learn-2_students/lectures/lecture8_advanced-deep-learning.html#an-example-generating-bitmojis) on how to do this.

In [ ]:
# Your solution goes here.

### 2.5: Train your GAN

You now have all the ingredients you need now to train a GAN - give it a go! Feel free to refer to the training code we came up with in [Lecture 8](https://pages.github.ubc.ca/MDS-2020-21/DSCI_572_sup-learn-2_students/lectures/lecture8_advanced-deep-learning.html#an-example-generating-bitmojis). You should track the loss of your model as epochs progress and show at least one example of an image output by your trained generator (better yet, record the evolution over time of how your generator is doing - just like we did in Lecture 8). Your results may not be great and that's okay, you should just show *something*. Here are some tips:
- You will likely need to train for at least `NUM_EPOCHS=100` (and maybe more).
- I find that the hardest part about training GANs is that the discriminator "overpowers" the generator, making it hard for the generator to learn how to create realistic images. There are lots of things you can do to try and balance your generator and discriminator, such as: play with the optimizer's hyperparameters, change the architectures of your models, etc.
- Here's a good set of [tips and tricks for training GANs](https://github.com/soumith/ganhacks).

>Once again, GANs are extremely difficult to train (even more so with smaller data sets like we have here). Don't worry if you're not getting amazing results - this is all about practice!

In [ ]:
# Your solution goes here.

## (Optional) Exercise 3: Setting Priorities and Managing your Time
<hr>

rubric={accuracy:1000000}

Admission into the MDS program is very competitive. This is great because we get to work with super motivated and talented individuals. However, such individuals often also have unrealistically high expectations of themselves. For example, some students feel they must do all the optional questions in a lab, even if they aren't particularly interested in them or don't really have the time to do them. In this lab, you only have to do one of the "mini-projects", you'll still get the solutions for the other one - so is it worth your time? If feeling obligated to do extra work sounds like you, this optional exercise is for you! You should not attempt (let alone complete!) this exercise. In skipping it, you will need to forego the 1,000,000 bonus points, but the hope is that you will feel better about skipping optional lab questions: nothing bad happens! Remember, MDS lab assignments are capped at 100% anyway. Doing everything perfectly is simply not possible forever, and when that time comes, it is important that you can set priorities. With that said, here's the task:

Provide video evidence of the following:

- 1,000 push-ups;
- 1,000 sit-ups;
- Learn to tie a bow tie (no pre-ties allowed!); and,
- Sing the entire Hamilton soundtrack from memory and with passion!

![](img/right.png)

>This question was inspired by a similar question Mike previously included in an MDS lab.

In [ ]:
# Your solution goes here.

## Submit to Canvas and GitHub
<hr>

When you are ready to submit your assignment do the following:
1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Run All Cells...`
2. Save your notebook.
3. Convert your notebook to `.html` format using the `convert_notebook()` function below or by `File -> Export Notebook As... -> Export Notebook to HTML`
4. Run the code `submit()` below to go through an interactive submission process to Canvas.
5. Finally, push all your work to GitHub (including the rendered html file).

In [ ]:
# convert_notebook("lab4.ipynb", "html")  # save your notebook, then uncomment and run when you want to convert to html

In [ ]:
# submit(course_code=59090)  # uncomment and run when ready to submit to Canvas